# Personalized Book Recommendation with CrewAI

This notebook demonstrates how to use CrewAI with Mem0 memory to:
1. Create personalized book recommendations based on user preferences
2. Store recommendation history for future context


## 1. Import Required Libraries


In [ ]:
from crewai import Agent, Crew, Task, Process
from mem0 import MemoryClient
import os

from dotenv import load_dotenv
load_dotenv()


## 2. Define the Recommendation Function


In [ ]:
def get_recommendation(user_id: str, user_preferences: str) -> str:
    recommendation_agent = Agent(
        role="Book Recommendation",
        backstory="Ai-powered book advisor who suggests books based on user preferences",
        goal="Provide personalised book recommendations.",
        verbose=True,
    )

    suggest_book_task = Task(
        description=f"Suggest three personalised book recommendations based on user preferences: {user_preferences}.",
        expected_output="A list of three book titles with brief description",
        agent=recommendation_agent
    )

    crew = Crew(
        agents=[recommendation_agent],
        tasks=[suggest_book_task],
        verbose=True,
        process=Process.sequential,
        memory=True,
        memory_config={
            "provider": "mem0",
            "config": {"user_id": user_id, "api_key": "m0-WdLTRzEc7XYRX6KLGpPmWYY8KaxuKBeuYRYPn7xo"},
        }
    )

    result = crew.kickoff()

    print("Final Recommendation:")
    print(result.raw)

    return result.raw


## 3. Define the Storage Function


In [ ]:
def store_user_recommendation(client: MemoryClient, user_id: str, conversation: list):
    client.add(conversation, user_id=user_id)


## 4. Initialize Memory Client


In [ ]:
client = MemoryClient(api_key="m0-WdLTRzEc7XYRX6KLGpPmWYY8KaxuKBeuYRYPn7xo")


## 5. Get User Input and Generate Recommendations


In [ ]:
# Set user ID and preferences
user_id = "user_123"  # Replace with your user ID
user_preferences = "Science fiction books with time travel themes"  # Replace with your preferences

# Get recommendations
result = get_recommendation(user_id, user_preferences)


## 6. Store Recommendation in Memory


In [ ]:
# Create conversation record
conversation = [
    {
        "role": "user",
        "content": f"Recommendation for {user_preferences}"
    },
    {
        "role": "assistant",
        "content": result
    }
]

# Store the recommendation in memory
store_user_recommendation(client, user_id, conversation)
print("Recommendation stored successfully!")
